In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import os
from sklearn.impute import SimpleImputer

In [2]:
import sagemaker

session = sagemaker.Session()

## Import Data

In [3]:
!unzip -o data/winemag-data-130k-v2.csv.zip
!mv winemag-data-130k-v2.csv data/winemag-data-130k-v2.csv

Archive:  data/winemag-data-130k-v2.csv.zip
  inflating: winemag-data-130k-v2.csv  


In [4]:
data = pd.read_csv('data/winemag-data-130k-v2.csv')

## Add Features

In [5]:
def year_extractor(title):
    year = None
    title = title.split()
    for word in title:
        try:
            number = int(word)
            if number >= 2000 and number <=2017:
                year = number
        except:
            pass
    return year

In [6]:
data['year'] = data['title'].apply(year_extractor)

## General Data Cleaning

##### Cleaning steps with no risk of data leakage
1. Remove CSV index column
2. Remove duplicates
3. Remove columns w/ significant null values
4. Remove categorical columns w/ too many unique values
5. Remove intances where the label (price) is null

In [7]:
cleaned_data = data.copy()

In [8]:
# 1. Remove CSV index column
del cleaned_data['Unnamed: 0']

In [9]:
# 2. Drop duplicate rows
cleaned_data.drop_duplicates(inplace=True)

In [10]:
# 3a. Remove columns with more than 50% missing values
del cleaned_data['region_2']

# 3b. Removing columns with more than 20% missing values that also wouldn't be available to the average wine shopper
del cleaned_data['taster_twitter_handle']
del cleaned_data['taster_name']

In [11]:
# 4. Remove columns with too many unique values
del cleaned_data['description']
del cleaned_data['title']
del cleaned_data['designation']
del cleaned_data['winery']

In [12]:
# 5. drop rows where price is null
cleaned_data = cleaned_data[cleaned_data['price'].notnull()]

## Split Data

In [13]:
# Separate labels and features
Y_labels = cleaned_data[['price']]
X_features = cleaned_data.copy()
del X_features['price']

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_features, Y_labels, test_size=0.10)
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.10)

## Sensative Data Cleaning

1. Remove outlying price values
2. Inputate missing date values
3. Replace rare countries with other
4. Replace rare provinces with other
5. Replace rare regions with other
4. Replace rare varieties wih other

In [14]:
# Recombine features and labels (to be split again later)
train = pd.concat([X_train, Y_train], axis=1)
val = pd.concat([X_val, Y_val], axis=1)
test = pd.concat([X_test, Y_test], axis=1)

### Train

In [15]:
train['price'].describe()

count    90389.000000
mean        35.560820
std         42.126246
min          4.000000
25%         17.000000
50%         25.000000
75%         42.000000
max       3300.000000
Name: price, dtype: float64

In [16]:
# 1. Remove outlying price values

# Calculate first and third quartile
first_quartile = train['price'].describe()['25%']
third_quartile = train['price'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove values outside of the first and third quartile
train = train[(train['price'] > (first_quartile - 3 * iqr)) & (train['price'] < (third_quartile + 3 * iqr))]

In [17]:
train['price'].describe()

count    88302.000000
mean        31.543612
std         20.089039
min          4.000000
25%         17.000000
50%         25.000000
75%         40.000000
max        116.000000
Name: price, dtype: float64

In [18]:
train['year'].isnull().sum()

4343

In [19]:
# 2. Imputate missing years with the most frequent value
train['year'].fillna(float(train['year'].mode()), inplace=True)

In [20]:
impute_year = train['year'].mode().iloc[0]
train['year'].isnull().sum()

0

In [21]:
train['country'].nunique()

42

In [22]:
# 3. Replace countries of low incidence with other 
types = train['country'].value_counts(normalize=True)
types = list(types[types.values < 0.001].index)
train['country'] = train['country'].replace(types, 'other')
train['country'] = train['country'].fillna('other')

In [23]:
country_list = list(train['country'].unique())
train['country'].nunique()

18

In [24]:
train['province'].nunique()

407

In [25]:
# 4. Replace provinces of low incidence with other
types = train['province'].value_counts(normalize=True)
types = list(types[types.values < 0.001].index)
train['province'] = train['province'].replace(types, 'other')
train['province'] = train['province'].fillna('other')

In [26]:
province_list = list(train['province'].unique())
train['province'].nunique()

74

In [27]:
train['region_1'].nunique()

1140

In [28]:
# 5. Replace regions of low incidence with other
countries_w_regions = ['US','Italy','France','Australia','Argentina','Spain','Canada']
regions_to_other = []

for country in countries_w_regions:
    regions = train[train['country']==country]['region_1'].value_counts(normalize=True)
    regions = list(regions[regions.values < 0.01].index)
    regions_to_other = regions_to_other + regions
    
train['region_1'] = train['region_1'].replace(regions_to_other, 'other')
train['region_1'].fillna('other', inplace=True)

In [29]:
region_list = list(train['region_1'].unique())
train['region_1'].nunique()

130

In [30]:
train['variety'].nunique()

658

In [31]:
types = train['variety'].value_counts(normalize=True)
types = list(types[types.values < 0.005].index)
train['variety'] = train['variety'].replace(types, 'other')
train['variety'] = train['variety'].fillna('other')

In [32]:
variety_list = list(train['variety'].unique())
train['variety'].nunique()

35

### Validation

In [33]:
# 1. Remove outlying price values

# Calculate first and third quartile
first_quartile = val['price'].describe()['25%']
third_quartile = val['price'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove values outside of the first and third quartile
val = val[(val['price'] > (first_quartile - 3 * iqr)) & (val['price'] < (third_quartile + 3 * iqr))]

In [34]:
# 2. Imputate missing years with the most frequent value from the training set
val['year'].fillna(float(impute_year), inplace=True)

In [35]:
def country_replace(value):
    if value not in country_list:
        return 'other'
    else:
        return value

In [36]:
# 3. Replace countries of low incidence with other 
val['country'] = val['country'].apply(country_replace)

In [37]:
def province_replace(value):
    if value not in province_list:
        return 'other'
    else:
        return value

In [38]:
# 4. Replace provinces of low incidence with other
val['province'] = val['province'].apply(province_replace)

In [39]:
def region_replace(value):
    if value not in region_list:
        return 'other'
    else:
        return value

In [40]:
# 5. Replace regions of low incidence with other
val['region_1'] = val['region_1'].apply(region_replace)

In [41]:
def variety_replace(value):
    if value not in variety_list:
        return 'other'
    else:
        return value

In [42]:
# 6. Replace varieties of low incidence with other
val['variety'] = val['variety'].apply(variety_replace)

### Test

In [43]:
# 1. Remove outlying price values

# Calculate first and third quartile
first_quartile = test['price'].describe()['25%']
third_quartile = test['price'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove values outside of the first and third quartile
test = test[(test['price'] > (first_quartile - 3 * iqr)) & (test['price'] < (third_quartile + 3 * iqr))]

In [44]:
# 2. Imputate missing years with the most frequent value
test['year'].fillna(float(impute_year), inplace=True)

In [45]:
# 3. Replace countries of low incidence with other 
test['country'] = test['country'].apply(country_replace)

In [46]:
# 4. Replace provinces of low incidence with other
test['province'] = test['province'].apply(province_replace)

In [47]:
# 5. Replace regions of low incidence with other
test['region_1'] = test['region_1'].apply(region_replace)

In [48]:
# 6. Replace varieties of low incidence with other
test['variety'] = test['variety'].apply(variety_replace)

In [49]:
train.head()

,country,points,province,region_1,variety,year,price
77080,US,93,California,Sta. Rita Hills,Pinot Noir,2014.0,30.0
116565,US,88,Oregon,Willamette Valley,Gewürztraminer,2013.0,18.0
28283,Italy,87,Northeastern Italy,Alto Adige,Grüner Veltliner,2009.0,28.0
93250,US,92,Washington,Walla Walla Valley (WA),Syrah,2006.0,40.0
29827,US,88,California,Sonoma Coast,Pinot Noir,2015.0,48.0


In [50]:
val.head()

,country,points,province,region_1,variety,year,price
75610,Italy,85,Northeastern Italy,Collio,other,2008.0,19.0
1255,South Africa,88,Stellenbosch,other,Merlot,2007.0,15.0
91585,Argentina,83,Mendoza Province,Uco Valley,Malbec,2014.0,18.0
58661,US,89,California,Sonoma County,Chardonnay,2014.0,65.0
28596,Italy,83,Piedmont,other,Barbera,2006.0,16.0


In [51]:
test.head()

,country,points,province,region_1,variety,year,price
40023,Chile,87,Casablanca Valley,other,Rosé,2015.0,12.0
91188,Australia,85,South Australia,Padthaway,Sparkling Blend,2013.0,17.0
108721,US,92,Washington,Walla Walla Valley (WA),Cabernet Sauvignon,2014.0,75.0
89602,US,87,Oregon,Willamette Valley,Pinot Noir,2013.0,36.0
43373,Spain,84,Northern Spain,Rueda,other,2000.0,13.0


## Helper functions & Values

In [52]:
def label_splitter(df):
    
    # splits a dataframe into features and label dataframes
    
    df_copy = df.copy()
    Y_df = df_copy[['price']]
    del df_copy['price']
    X_df = df_copy
    
    return X_df, Y_df

In [53]:
def one_hot_encode_column(df, column):
    df = pd.concat([df, pd.get_dummies(df[column])], axis=1)
    del df[column]
    
    return df

In [54]:
# Create data directory if it doesn't exist
data_dir = '../SageMaker/data/wine'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Test 1

Train an inital model using only features: points, price, year (vintage), country and variety

In [55]:
# Select initial features
train1 = train[['points','price','year','country','variety']]
val1 = val[['points','price','year','country','variety']]
test1 = test[['points','price','year','country','variety']]

In [56]:
X_train1, Y_train1 = label_splitter(train1)
X_val1, Y_val1 = label_splitter(val1)
X_test1, Y_test1 = label_splitter(test1)

#### Feature Engineering

In [57]:
# 1. One-hot encode country
X_train1 = one_hot_encode_column(X_train1, 'country')
X_val1 = one_hot_encode_column(X_val1, 'country')
X_test1 = one_hot_encode_column(X_test1, 'country')

In [58]:
# 2. One-hot encode variety
X_train1 = one_hot_encode_column(X_train1, 'variety')
X_val1 = one_hot_encode_column(X_val1, 'variety')
X_test1 = one_hot_encode_column(X_test1, 'variety')

In [59]:
X_train1.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
77080,93,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116565,88,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28283,87,2009.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93250,92,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29827,88,2015.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
X_val1.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
75610,85,2008.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1255,88,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91585,83,2014.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58661,89,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28596,83,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
X_test1.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
40023,87,2015.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
91188,85,2013.0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
108721,92,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89602,87,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43373,84,2000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Upload to S3

In [62]:
# Save test and train data to csvs
X_test1.to_csv(os.path.join(data_dir, 'test1.csv'), header=False, index=False)
Y_test1.to_csv(os.path.join(data_dir, 'test1_labels.csv'), header=False, index=False)
pd.concat([Y_train1, X_train1], axis=1).to_csv(os.path.join(data_dir, 'train1.csv'), header=False, index=False)
pd.concat([Y_val1, X_val1], axis=1).to_csv(os.path.join(data_dir, 'validation1.csv'), header=False, index=False)

In [63]:
prefix = 'capstone-wine'

test_location = session.upload_data(os.path.join(data_dir, 'test1.csv'), key_prefix=prefix)
test_labels_location = session.upload_data(os.path.join(data_dir, 'test1_labels.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train1.csv'), key_prefix=prefix)
validation_location = session.upload_data(os.path.join(data_dir, 'validation1.csv'), key_prefix=prefix)

print(test_location)
print(test_labels_location)
print(train_location)
print(validation_location)

s3://sagemaker-eu-west-2-115849693658/capstone-wine/test1.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/test1_labels.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/train1.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/validation1.csv


## Test 2

Train a model using only features: points, price, year (vintage), region and variety

In [64]:
# Select initial features
train2 = train[['points','price','year','region_1','variety']]
val2 = val[['points','price','year','region_1','variety']]
test2 = test[['points','price','year','region_1','variety']]

In [65]:
X_train2, Y_train2 = label_splitter(train2)
X_val2, Y_val2 = label_splitter(val2)
X_test2, Y_test2 = label_splitter(test2)

#### Feature Engineering

In [66]:
def one_hot_dummy_columns(df, train_value_list, comparison_df, column):
    comparison_list = set(comparison_df[column].unique())
    missing_columns = list(set(train_value_list) - comparison_list)
    df[missing_columns] = pd.DataFrame([len(missing_columns)*[0]], index=df.index)
    return df

In [67]:
# 1. One-hot encode region
X_train2 = one_hot_encode_column(X_train2, 'region_1')
X_val2 = one_hot_encode_column(X_val2, 'region_1')
X_val2 = one_hot_dummy_columns(X_val2, region_list, val2, 'region_1')
X_test2 = one_hot_encode_column(X_test2, 'region_1')
X_test2 = one_hot_dummy_columns(X_test2, region_list, test2, 'region_1')

In [68]:
# 2. One-hot encode variety
X_train2 = one_hot_encode_column(X_train2, 'variety')
X_val2 = one_hot_encode_column(X_val2, 'variety')
X_val2 = one_hot_dummy_columns(X_val2, variety_list, val2, 'variety')
X_test2 = one_hot_encode_column(X_test2, 'variety')
X_test2 = one_hot_dummy_columns(X_test2, variety_list, test2, 'variety')

In [69]:
X_train2.head()

,points,year,Adelaide Hills,Agrelo,Alexander Valley,Alicante,Alsace,Alto Adige,Amarone della Valpolicella Classico,Anderson Valley,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
77080,93,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116565,88,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28283,87,2009.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
93250,92,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29827,88,2015.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
X_val2.head()

,points,year,Adelaide Hills,Agrelo,Alexander Valley,Alicante,Alsace,Alto Adige,Amarone della Valpolicella Classico,Anderson Valley,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
75610,85,2008.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1255,88,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91585,83,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58661,89,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28596,83,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
X_test2.head()

,points,year,Adelaide Hills,Agrelo,Alexander Valley,Alicante,Alsace,Alto Adige,Amarone della Valpolicella Classico,Anderson Valley,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
40023,87,2015.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91188,85,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
108721,92,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89602,87,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43373,84,2000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Upload to S3

In [72]:
# Save test and train data to csvs
X_test2.to_csv(os.path.join(data_dir, 'test2.csv'), header=False, index=False)
Y_test2.to_csv(os.path.join(data_dir, 'test2_labels.csv'), header=False, index=False)
pd.concat([Y_train2, X_train2], axis=1).to_csv(os.path.join(data_dir, 'train2.csv'), header=False, index=False)
pd.concat([Y_val2, X_val2], axis=1).to_csv(os.path.join(data_dir, 'validation2.csv'), header=False, index=False)

In [73]:
prefix = 'capstone-wine'

test_location = session.upload_data(os.path.join(data_dir, 'test2.csv'), key_prefix=prefix)
test_labels_location = session.upload_data(os.path.join(data_dir, 'test2_labels.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train2.csv'), key_prefix=prefix)
validation_location = session.upload_data(os.path.join(data_dir, 'validation2.csv'), key_prefix=prefix)

print(test_location)
print(test_labels_location)
print(train_location)
print(validation_location)

s3://sagemaker-eu-west-2-115849693658/capstone-wine/test2.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/test2_labels.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/train2.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/validation2.csv


## Test 3

Train a model using all features: points, price, year (vintage), country, province, region and variety

In [74]:
# Select initial features
train3 = train.copy()
val3 = val.copy()
test3 = test.copy()

In [75]:
X_train3, Y_train3 = label_splitter(train3)
X_val3, Y_val3 = label_splitter(val3)
X_test3, Y_test3 = label_splitter(test3)

#### Feature Engineering

In [76]:
# 1. One-hot encode country
X_train3 = one_hot_encode_column(X_train3, 'country')
X_val3 = one_hot_encode_column(X_val3, 'country')
X_val3 = one_hot_dummy_columns(X_val3, country_list, val3, 'country')
X_test3 = one_hot_encode_column(X_test3, 'country')
X_test3 = one_hot_dummy_columns(X_test3, country_list, test3, 'country')

In [77]:
# 2. One-hot encode region
X_train3 = one_hot_encode_column(X_train3, 'region_1')
X_val3 = one_hot_encode_column(X_val3, 'region_1')
X_val3 = one_hot_dummy_columns(X_val3, region_list, val3, 'region_1')
X_test3 = one_hot_encode_column(X_test3, 'region_1')
X_test3 = one_hot_dummy_columns(X_test3, region_list, test3, 'region_1')

In [78]:
# 3. One-hot encode province
X_train3 = one_hot_encode_column(X_train3, 'province')
X_val3 = one_hot_encode_column(X_val3, 'province')
X_val3 = one_hot_dummy_columns(X_val3, province_list, val3, 'province')
X_test3 = one_hot_encode_column(X_test3, 'province')
X_test3 = one_hot_dummy_columns(X_test3, province_list, test3, 'province')

In [79]:
# 4. One-hot encode variety
X_train3 = one_hot_encode_column(X_train3, 'variety')
X_val3 = one_hot_encode_column(X_val3, 'variety')
X_val3 = one_hot_dummy_columns(X_val3, variety_list, val3, 'variety')
X_test3 = one_hot_encode_column(X_test3, 'variety')
X_test3 = one_hot_dummy_columns(X_test3, variety_list, test3, 'variety')

In [80]:
X_train3.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
77080,93,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116565,88,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28283,87,2009.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93250,92,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29827,88,2015.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
X_val3.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
75610,85,2008.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1255,88,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91585,83,2014.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58661,89,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28596,83,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
X_test3.head()

,points,year,Argentina,Australia,Austria,Bulgaria,Canada,Chile,France,Germany,...,Sangiovese,Sauvignon Blanc,Shiraz,Sparkling Blend,Syrah,Tempranillo,Viognier,White Blend,Zinfandel,other
40023,87,2015.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
91188,85,2013.0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
108721,92,2014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89602,87,2013.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43373,84,2000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Upload to S3

In [83]:
# Save test and train data to csvs
X_test3.to_csv(os.path.join(data_dir, 'test3.csv'), header=False, index=False)
Y_test3.to_csv(os.path.join(data_dir, 'test3_labels.csv'), header=False, index=False)
pd.concat([Y_train3, X_train3], axis=1).to_csv(os.path.join(data_dir, 'train3.csv'), header=False, index=False)
pd.concat([Y_val3, X_val3], axis=1).to_csv(os.path.join(data_dir, 'validation3.csv'), header=False, index=False)

In [84]:
prefix = 'capstone-wine'

test_location = session.upload_data(os.path.join(data_dir, 'test3.csv'), key_prefix=prefix)
test_labels_location = session.upload_data(os.path.join(data_dir, 'test3_labels.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train3.csv'), key_prefix=prefix)
validation_location = session.upload_data(os.path.join(data_dir, 'validation3.csv'), key_prefix=prefix)

print(test_location)
print(test_labels_location)
print(train_location)
print(validation_location)

s3://sagemaker-eu-west-2-115849693658/capstone-wine/test3.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/test3_labels.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/train3.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/validation3.csv


## Test 4

Train model using hyperparameter tuner

In [85]:
X_test3.to_csv(os.path.join(data_dir, 'test4.csv'), header=False, index=False)
Y_test3.to_csv(os.path.join(data_dir, 'test4_labels.csv'), header=False, index=False)

In [86]:
prefix = 'capstone-wine'

test_location = session.upload_data(os.path.join(data_dir, 'test4.csv'), key_prefix=prefix)
test_labels_location = session.upload_data(os.path.join(data_dir, 'test4_labels.csv'), key_prefix=prefix)

print(test_location)
print(test_labels_location)

s3://sagemaker-eu-west-2-115849693658/capstone-wine/test4.csv
s3://sagemaker-eu-west-2-115849693658/capstone-wine/test4_labels.csv


## Save CSV w/ Header For Evaluation

In [89]:
X_test3.to_csv(os.path.join(data_dir, 'test4_header.csv'))
Y_test3.to_csv(os.path.join(data_dir, 'test4_labels_header.csv'))